<a href="https://colab.research.google.com/github/GLuarte/TareasX/blob/master/Fixture%20Futbol%20(TareaSofi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

In [ ]:
from pulp import *
import numpy as np


In [ ]:

a=4

#m=3
#n=5

#equipos_m=[]
#for i in range (1,m+1):
#  eq='Equipo '+str(i)
#  equipos_m.append(eq)

#equipos_n=[]
#for i in range (m+1,m+n+1):
#  eq='Equipo '+str(i)
#  equipos_n.append(eq)

#equipos=equipos_m+equipos_n

fechas=[]
for i in range (1,m+n):
  eq='Fecha '+str(i)
  fechas.append(eq)

In [ ]:
equipos_m=['Colo Colo','U de Chile','U Catolica']
equipos_n=['Dep Iquique','Cobresal',"O'Higgins",'Huachipato','UdeC']
equipos=equipos_m+equipos_n

In [ ]:
torneo=np.zeros((len(fechas),len(equipos),len(equipos)))

In [ ]:
x=LpVariable.dicts('Partido',
                   [(i,j,k) for i in equipos for j in equipos for k in fechas],
                   lowBound=0,
                   upBound=1,
                   cat='Integer'
                   )

In [ ]:
z=LpProblem('Fixture Futbol',LpMaximize)

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1190: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
z+=lpSum((x[(i,j,k)])for k in fechas for i in equipos for j in equipos)

In [ ]:
# No se puede jugar contra si mismo
for i in equipos:
  for k in fechas:
    z+=x[(i,i,k)]==0

In [ ]:
#Todos deben jugar contra todos
for i in equipos:
  for j in equipos:
    z+=lpSum((x[(i,j,k)]+x[(j,i,k)])for k in fechas)<=1

In [ ]:
#cada equipo debe jugar 1 partido x fecha
for j in equipos:
  for k in fechas:
    z+=lpSum((x[(i,j,k)]+x[(j,i,k)]) for i in equipos)==1

In [ ]:
#se deben juar un maximo de 'a' partidos como local
for i in equipos:
  z+=lpSum((x[(i,j,k)])for k in fechas for j in equipos )<=a

In [ ]:
#se deben priorizar los clasicos para las ultimas fechas
z+=lpSum((x[(i,j,k)]+x[(j,i,k)])for i in equipos_m for j in equipos_m for k in fechas[:3])==0

In [ ]:
#Los equipos pequeños deben jugar a lo menos 1 partido de local
z+=lpSum((x[(i,j,k)])for i in equipos_n for j in equipos_m for k in fechas)>=1

In [ ]:
z.writeLP('tareaSofi.lp')
z.solve()
print('status:', LpStatus[z.status])
#for v in z.variables():
#  print(v.name, v.varValue)
print ('FO:',value(z.objective))

status: Optimal
FO: 28.0


In [ ]:
for n,k in enumerate(fechas):
  for m,j in enumerate(equipos):
    for l,i in enumerate(equipos):
      torneo[n,l,m]=float(x[(i,j,k)].varValue)
      if torneo[n,l,m]==1:
        print(k,i,'vs',j)
  print('\n')

Fecha 1 Dep Iquique vs Cobresal
Fecha 1 Colo Colo vs O'Higgins
Fecha 1 U de Chile vs Huachipato
Fecha 1 U Catolica vs UdeC


Fecha 2 Huachipato vs Colo Colo
Fecha 2 UdeC vs Dep Iquique
Fecha 2 U Catolica vs Cobresal
Fecha 2 U de Chile vs O'Higgins


Fecha 3 UdeC vs Colo Colo
Fecha 3 Cobresal vs U de Chile
Fecha 3 O'Higgins vs U Catolica
Fecha 3 Dep Iquique vs Huachipato


Fecha 4 U Catolica vs Colo Colo
Fecha 4 U de Chile vs Dep Iquique
Fecha 4 Huachipato vs Cobresal
Fecha 4 O'Higgins vs UdeC


Fecha 5 Colo Colo vs U de Chile
Fecha 5 Dep Iquique vs U Catolica
Fecha 5 O'Higgins vs Cobresal
Fecha 5 UdeC vs Huachipato


Fecha 6 UdeC vs U de Chile
Fecha 6 Colo Colo vs Cobresal
Fecha 6 Dep Iquique vs O'Higgins
Fecha 6 U Catolica vs Huachipato


Fecha 7 U de Chile vs U Catolica
Fecha 7 Colo Colo vs Dep Iquique
Fecha 7 Huachipato vs O'Higgins
Fecha 7 Cobresal vs UdeC


